In [1]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf

2023-05-22 13:07:31.083729: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
dev = qml.device('cirq.simulator',wires=3)

In [3]:
def real(angles,**kwargs):
    qml.Hadamard(wires=0)
    qml.Rot(*angles,wires=0)

In [4]:
#drawer = qml.draw(real)
#w = ['phi', 'theta', 'omega']
#print(drawer(w))

In [5]:
def generator(w,**kwargs):
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=1)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=1)
    qml.CNOT(wires=[0,1])
    qml.RX(w[6], wires=0)
    qml.RY(w[7], wires=0)
    qml.RZ(w[8], wires=0)

In [6]:
#drawer = qml.draw(generator,show_all_wires=True,wire_order=[0,1,2])
#w = ['w0','w1','w2','w3','w4','w5','w6','w7','w8']
#print(drawer(w))

In [7]:
def discriminator(w):
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=2)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=2)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=2)
    qml.CNOT(wires=[0, 2])
    qml.RX(w[6], wires=2)
    qml.RY(w[7], wires=2)
    qml.RZ(w[8], wires=2)

In [8]:
#drawer = qml.draw(discriminator,show_all_wires=True,wire_order=[0,1,2])
#w = ['w0','w1','w2','w3','w4','w5','w6','w7','w8']
#print(drawer(w))

In [9]:
@qml.qnode(dev, interface='tf')
def real_disc_circuit(phi, theta, omega, disc_weights):
    real([phi,theta,omega])
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2))

@qml.qnode(dev, interface='tf')
def gen_disc_circuit(gen_weights,disc_weights):
    generator(gen_weights)
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2))

In [10]:
def prob_real_true(disc_weights):
    true_disc_output = real_disc_circuit(phi,theta,omega,disc_weights)
    #convert to probability
    prob_real_true = (true_disc_output+1)/2
    return prob_real_true

In [11]:
def prob_fake_true(gen_weights,disc_weights):
    fake_disc_output = gen_disc_circuit(gen_weights,disc_weights)
    #convert to probability
    prob_fake_true = (fake_disc_output+1)/2
    return prob_fake_true

In [12]:
def disc_cost(disc_weights):
    cost = prob_fake_true(gen_weights,disc_weights)-prob_real_true(disc_weights)
    return cost
def gen_cost(gen_weights):
    return -prob_fake_true(gen_weights,disc_weights)

In [13]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 8) + \
                   np.random.normal(scale=eps, size=(9,))
init_disc_weights = np.random.normal(size=(9,))

gen_weights = tf.Variable(init_gen_weights)
disc_weights = tf.Variable(init_disc_weights)

2023-05-22 13:07:41.731373: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
opt = tf.keras.optimizers.SGD(0.4)

In [15]:
cost = lambda: disc_cost(disc_weights)

for step in range(50):
    opt.minimize(cost, disc_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute '_unique_id'